In [ ]:
!pip install rdkit
!pip install deepchem
!pip install tensorflow
!pip install scikit-learn pandas numpy matplotlib seaborn shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.4/552.4 kB 17.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

import deepchem as dc
from deepchem.feat import ConvMolFeaturizer
from deepchem.models import GraphConvModel

#

def load_bace_dataset():
    print("Loading BACE dataset from DeepChem...")
    tasks, datasets, transformers = dc.molnet.load_bace_classification()
    train, val, test = datasets

    def dc_to_df(dc_dataset):
        smiles = dc_dataset.ids
        y = dc_dataset.y.flatten()
        return pd.DataFrame({"smiles": smiles, "y": y})

    df_train = dc_to_df(train)
    df_val = dc_to_df(val)
    df_test = dc_to_df(test)

    df = pd.concat([df_train, df_val, df_test], ignore_index=True)
    print("Dataset loaded:", df.shape)
    return df


# Load & Clean Data

def load_and_clean_csv(path="data.csv", smiles_col="smiles", label_col="y"):
    df = load_bace_dataset()
    df = df[[smiles_col, label_col]].dropna()
    df.rename(columns={smiles_col: "smiles", label_col: "y"}, inplace=True)

    mols = []
    valid_idx = []
    for i, smi in enumerate(df["smiles"]):
        mol = Chem.MolFromSmiles(smi)
        if mol is not None:
            mols.append(mol)
            valid_idx.append(i)

    df = df.iloc[valid_idx].reset_index(drop=True)
    df["mol"] = mols
    df["y"] = df["y"].astype(int)
    return df


def make_splits(df, test_size=0.2, val_size=0.2, random_state=42):
    idx = np.arange(len(df))
    train_idx, test_idx = train_test_split(
        idx, test_size=test_size, random_state=random_state, stratify=df["y"]
    )
    train_idx, val_idx = train_test_split(
        train_idx, test_size=val_size, random_state=random_state, stratify=df["y"].iloc[train_idx]
    )
    return train_idx, val_idx, test_idx


# PIPELINE A — RDKIT DESCRIPTORS

def compute_rdkit_descriptors(mols):
    desc_names = [d[0] for d in Descriptors._descList]
    desc_funcs = [d[1] for d in Descriptors._descList]

    data = []
    for mol in mols:
        row = []
        for f in desc_funcs:
            try:
                row.append(f(mol))
            except Exception:
                row.append(np.nan)
        data.append(row)

    df_desc = pd.DataFrame(data, columns=desc_names)
    # NaN value
    df_desc = df_desc.replace([np.inf, -np.inf], np.nan)
    df_desc = df_desc.dropna(axis=1)  #drop Nan
    return df_desc


def run_rdkit_pipeline(df, train_idx, val_idx, test_idx):
    print("\n=== Pipeline A: RDKit Descriptors + RandomForest ===")
    X_all = compute_rdkit_descriptors(df["mol"].tolist())
    y_all = df["y"].values

    # remove low-variance
    vt = VarianceThreshold(threshold=0.0)
    X_all = vt.fit_transform(X_all)

    scaler = StandardScaler()
    X_all = scaler.fit_transform(X_all)

    X_train, X_val, X_test = X_all[train_idx], X_all[val_idx], X_all[test_idx]
    y_train, y_val, y_test = y_all[train_idx], y_all[val_idx], y_all[test_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        n_jobs=-1,
        random_state=42
    )
    clf.fit(X_train, y_train)

    def auc(split_name, X, y):
        proba = clf.predict_proba(X)[:, 1]
        score = roc_auc_score(y, proba)
        print(f"{split_name} ROC-AUC = {score:.3f}")
        return score

    auc("Train", X_train, y_train)
    auc("Valid", X_val, y_val)
    auc("Test ", X_test, y_test)

    return clf


# PIPELINE B — ECFP4 FINGERPRINTS

def compute_ecfp4_bits(mols, n_bits=1024, radius=2):
    fps = []
    for mol in mols:
        bv = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits)
        arr = np.zeros((n_bits,), dtype=int)
        Chem.DataStructs.ConvertToNumpyArray(bv, arr)
        fps.append(arr)
    return np.array(fps)


def run_ecfp4_pipeline(df, train_idx, val_idx, test_idx):
    print("\n=== Pipeline B: ECFP4 (1024-bit) + LogisticRegression ===")
    X_all = compute_ecfp4_bits(df["mol"].tolist(), n_bits=1024, radius=2)
    y_all = df["y"].values

    scaler = StandardScaler()
    X_all = scaler.fit_transform(X_all)

    X_train, X_val, X_test = X_all[train_idx], X_all[val_idx], X_all[test_idx]
    y_train, y_val, y_test = y_all[train_idx], y_all[val_idx], y_all[test_idx]

    clf = LogisticRegression(
        max_iter=2000,
        n_jobs=-1,
        random_state=42
    )
    clf.fit(X_train, y_train)

    def auc(split_name, X, y):
        proba = clf.predict_proba(X)[:, 1]
        score = roc_auc_score(y, proba)
        print(f"{split_name} ROC-AUC = {score:.3f}")
        return score

    auc("Train", X_train, y_train)
    auc("Valid", X_val, y_val)
    auc("Test ", X_test, y_test)

    return clf

# Comparison
if __name__ == "__main__":
    df = load_and_clean_csv("data.csv", smiles_col="smiles", label_col="y")
    print(f"Loaded {len(df)} molecules after cleaning.")

    train_idx, val_idx, test_idx = make_splits(df)

    # A: RDKit descriptors
    rdkit_model = run_rdkit_pipeline(df, train_idx, val_idx, test_idx)

    # B: ECFP4 fingerprints
    ecfp_model = run_ecfp4_pipeline(df, train_idx, val_idx, test_idx)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Loading BACE dataset from DeepChem...


[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerator
[02:35:08] DEPRECATION WARNING: please use MorganGenerat

Dataset loaded: (1513, 2)
Loaded 1513 molecules after cleaning.

=== Pipeline A: RDKit Descriptors + RandomForest ===
Train ROC-AUC = 1.000
Valid ROC-AUC = 0.872
Test  ROC-AUC = 0.877

=== Pipeline B: ECFP4 (1024-bit) + LogisticRegression ===


[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerator
[02:35:44] DEPRECATION WARNING: please use MorganGenerat

Train ROC-AUC = 1.000
Valid ROC-AUC = 0.796
Test  ROC-AUC = 0.853
